We compute in this worksheet the orbifold fundamental group $G_2$ of Proposition 4.3 in the paper `Torsion divisors of plane curves with maximal flexes and Zariski pairs` by E. Artal, S. Bannai, T. Shirane and H. Tokunaga. As for other groups we use the package `sirocco` by M.Á Marco and M. Rodríguez. Since we need to know which are the meridians of the irreducible components (including the line at infinity), instead of using the method `fundamental_group` we use `braid_monodromy`.

We introduce the curve. The equations are in $\mathbb{Q}$; $f$ is the cubic, $T_1$ is the equation of one triangle (which contains the line at infinity) and $T_2$ is the equation of another triangle (this polynomial has three linear factors in an extension of $\mathbb{Q}$).

In [2]:
S1.<x,y>=QQ[]
f=x^2*y+y^2+x
T1=x*y
T2=x^3 - 3*x*y^2 + y^3 - 3*x^2 - 3*x*y - 3*y + 1
F=(f*T1*T2)(x=x+y)

In [3]:
%%time
C=Curve(F)
L=C.braid_monodromy()

CPU times: user 782 ms, sys: 1.1 s, total: 1.88 s
Wall time: 8.92 s


In [4]:
len(L)

15

In [5]:
def trconm(L):
    A=[]
    for j in range(len(L)):
        if j not in A:
            L1=[k for k in range(j+1,len(L)) if L[j]+L[k]==0]
            if L1!=[]:
                k=L1[0]
                L2=[i for i in L[j+1:k] if abs(abs(i)-abs(L[j]))<2]
                if L2==[]:
                    A+=[j,k]
    return tuple(L[j] for j in range(len(L)) if j not in A)

def trconm2(L0):
    L1=[_ for _ in L0]
    L2=trconm(L1)
    while len(L2)<len(L1):
        L1=[_ for _ in L2]
        L2=trconm(L1)
    return(L2)

def romper(L0):
    c0=()
    b0=L0
    while b0[0]+b0[-1]==0:
        c0=c0+(b0[0],)
        b0=b0[1:-1]
    return (c0,b0)

In [6]:
F1=FreeGroup(8)
B=BraidGroup(8)
pares0=[]
for aux in L:
    aux=B(F1(aux.Tietze()).Tietze())
    aux=B(trconm2(aux.Tietze()))
    pares0.append(romper(aux.Tietze()))
puiseux=[]

We want to modify the above decomposition in order to have positive braids for the middle braids. The result will be kept in the list `puiseux`. Each element of the list `puiseux` have four entries: two braids $\tau_1,\tau_2$ ($\tau_2$ is a positive algebraic braid) and two numbers $m,n$. The braid associated to the basis element is $\tau_1\cdot\tau_2\cdot\tau_1^{-1}$; $m$ is the first strand involved in $\tau_2$ and $n$ is the number of strands involved in $\tau_2$.

In order to keep this computations as short as possible, we need to manipulate the braids one by one.

In [7]:
print(pares0[0])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, 4, 5, -4, 6, 3, 4, 5, 6, 2, 3, 7, 4, 6, 5), (6, 4, 7, 3, 6, -7, -6, -5, 4, -7, -3, 5, -4))


In [8]:
v=B((-6, -6, 5, 4, 3,))
w=B([2,])
u=L[0]
m=1
u==v*w^m/v

True

In [9]:
puiseux.append([v,w^m,2,2])

In [10]:
print(pares0[1])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, 4, 5, -4, 6, 3, 4, 5, 6, 2, 3, 7, 4, 6, 5, 4, -5, 3, 7, -4, 5, 6, 2, 5, 7, 4, 3, 6, -4), (5, 7, 6, 5, 5, 6, 5, 6, 6, -7))


In [11]:
v=B((1, 7, 6, 5, 4, 3, 6, 7, 5, 6, 7, 2, 7, 6, 5, 4, 3, -6, 7, -6, 5, 4, 4, 5, 6, 7,))
w=B([5, 6, 6, 5, 6, 5, 5, 6,])
u=L[1]
m=1
u==v*w^m/v

True

In [12]:
puiseux.append([v,w^m,5,3])

In [13]:
print(pares0[2])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, 4, 5, -4, 6, 3, 4, 5, 6, 2, 3, 7, 4, 6, 5, 4, -5, 3, 7, -4, 5, 6, 2, 5), (7, 4, 3, 6, -4, 7, -6, -2, -3, -2, -6, 7, 6, 7, -6, 3, 2, 3, -6, 4, -3, -4))


In [14]:
v=B((-6, -6, 5, 5, 4,))
w=B([3])
u=L[2]
m=2
u==v*w^m/v

True

In [15]:
puiseux.append([v,w^m,3,2])

In [16]:
print(pares0[3])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, 4, 5, -4, 6, 3, 4, 5, 6, 2, 3, 7, 4, 6, 5, 4, -5, 3, 7, -4, 5, 6, 2, 5), (4, 3, -4, -3, -2, -3, -4, 3, 4, 3, 2, 3))


In [17]:
v=B((1, -6, -6, -6, -5, 4, 2,))
w=B([3, ])
u=L[3]
m=2
u==v*w^m/v

True

In [18]:
puiseux.append([v,w^m,3,2])

In [19]:
print(pares0[4])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, 4, 5, -4, 6, 3, 4, 5, 6, 2, 3, 7, 4, 6, 5, 4, -5, 3, 7, -4), (5, 6, 2, 5, 6, -3, -2, -3, 6, 5, 2, 6, 5, 3))


In [20]:
v=B((1, 7, 6, 5, 4, 5, 3, 2, 5, 6, 7, 5, 4, 6, 5, -4, 3, 4, 5, 6, 5, 4, 6, 7))
w=B([6, 5, 6, 5, 6, 6, 6, 5])
u=L[4]
m=1
u==v*w^m/v

True

In [21]:
puiseux.append([v,w^m,5,3])

In [22]:
print(pares0[5])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, 4, 5, -4, 6, 3, 4, 5, 6, 2), (3, 7, 4, 6, 5, 4, -5, 3, 7, -4, -3, -4, 7, -5, -6, -7))


In [23]:
v=B((1,   ))
w=B([2])
u=L[5]
m=2
u==v*w^m/v

True

In [24]:
puiseux.append([v,w^m,2,2])

In [25]:
print(pares0[6])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 5, 6, 4, 5, -4, 6, 3, 4), (5, 6, 7, 6, 5, -7, 4, -5, -6, -5, -7))


In [26]:
v=B((-6, 4, 5,  ))
w=B([6])
u=L[6]
m=1
u==v*w^m/v

True

In [27]:
puiseux.append([v,w^m,6,2])

In [28]:
print(pares0[7])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6), (4, -7, 2, -6, -5, -4, 3, -5, 4, 5, -2, 6, 7))


In [29]:
v=B((1, 7, 6, 5, -3, 4, 1,  ))
w=B([  2,])
u=L[7]
m=1
u==v*w^m/v

True

In [30]:
puiseux.append([v,w^m,2,2])

In [31]:
print(pares0[8])

((1, 7, 6, 5, 4, 5, 3, 6, 7, 2, 1, 5, 6, -7, -6, 2, -5, -4, 5, -3), (4,))


In [32]:
v=B((1, 5, 4,  1, -2,))
w=B([  3,])
u=L[8]
m=1
u==v*w^m/v

True

In [33]:
puiseux.append([v,w^m,3,2])

In [34]:
print(pares0[9])

((7, 6, 5, 4, 5, 6), (7, 5, 6, -7, -6, -5, -6, -7, 6, 7, 6, 5, 6, -5))


In [35]:
v=B((-6, ))
w=B([ 5, ])
u=L[9]
m=2
u==v*w^m/v

True

In [36]:
puiseux.append([v,w^m,5,2])

In [37]:
print(pares0[10])

((), (1, 7, 6, 5, 4, 5, 3, 6, 2, 1, 5, -6, 2, -5, 2, -6, 5, 1, 2, 6, 5, -6, 1, -5, -6, -3, -4, -5, -6, -7))


In [38]:
v=B(( 7, 6, 5, 4, 3,))
w=B([1,  2, 1, 2, 2, 1, 2, 1,  ])
u=L[10]
m=1
u==v*w^m/v

True

In [39]:
puiseux.append([v,w^m,1,3])

In [40]:
print(pares0[11])

((7, 6, 5, 4), (3, 3))


In [41]:
v=B(( 7, 6, 5, 4, ))
w=B([3,  ])
u=L[11]
m=2
u==v*w^m/v

True

In [42]:
puiseux.append([v,w^m,1,3])

In [43]:
print(pares0[12])

((7, 6, 5), (4, 6, 5, 4, -5, -5, -6))


In [44]:
v=B(( 7, 6, 6, 5, ))
w=B([4 ])
u=L[12]
m=1
u==v*w^m/v

True

In [45]:
puiseux.append([v,w^m,4,2])

In [46]:
print(pares0[13])

((7,), (6, 5, 6, 5, 5, 6, 5, 6))


In [47]:
v=B(( 7,))
w=B([6, 5, 6, 5, 5, 6, 5, 6 ])
u=L[13]
m=1
u==v*w^m/v

True

In [48]:
puiseux.append([v,w^m,6,3])

In [49]:
print(pares0[14])

((), (7, 7))


In [50]:
v=B(( ))
w=B([7 ])
u=L[14]
m=2
u==v*w^m/v

True

In [51]:
puiseux.append([v,w^m,7,2])

The following cell shows the fifteen braids of the braid monodromy. We give to braids $\alpha,\beta$ and the braid is $\alpha\cdot\beta\cdot\alpha^{-1}$.

In [52]:
m=1
for i,j,k,l in puiseux:
    print(m)
    show(i)
    show(j)
    m+=1

1


s5^-2*s4*s3*s2

s1

2


s0*s6*s5*s4*s3*s2*s5*s6*s4*s5*s6*s1*s6*s5*s4*s3*s2*s5^-1*s6*s5^-1*s4*s3^2*s4*s5*s6

...

s6

s5*(s4*s5*s4)^2*s5

15


1

s6^2

We have all the Puiseux decompositions of the braids. We need to identify to which component each meridian belongs. We see that the generators $\mu_2,\mu_4,\mu_6$ correspond to the cubic. The other ones (and the meridian at infinity) correspond to the lines. We may distinguish the two triangles but we will do it later. This is done looking at the orbits by the monodromy action.

We start with the free group corresponding to the fundamental group of the complement of the curve in a vertical line. We add the orbifold relations: the cubes of the meridians of the lines and the 9th power of the meridian of the cubic.

In [55]:
permutaciones=PermutationGroup([_.permutation() for _ in L])
print(permutaciones.orbits())
FL8=FreeGroup(8)
L8=[FL8(_)^3 for _ in [[1], [3], [5], [7], [8], [1..8]]]+[FL8([2])^9]

[[1], [2, 4, 6], [3], [5], [7], [8]]


We add the relations of the braid monodromy using the Puiseux decomposition.

In [56]:
for tau1,tau2,m0,m1 in puiseux:
    for j in [m0..m0+m1-1]:
        v=((FL8([j])*tau2)/FL8([j]))*tau1^-1
        L8.append(v)

In [57]:
G=(FL8/L8)

We simplify the presentation and we keep track of the meridians. We see that this orbifold group is finite. As it is bigger than the abelianization, the group is not abelian.

In [58]:
hom1=G.simplification_isomorphism()
G2=hom1.codomain()

In [59]:
G2.order()

6561

In [60]:
ab=G2.abelian_invariants()
print (ab)
prod(ab)

(3, 3, 3, 3, 3, 9)


2187

We express the meridians in terms of the generators of $G_2$. We denote by $w$ a meridian of the cubic and we put the commutators of the generators in a list.

In [62]:
meridianos=[hom1(G(_)).Tietze() for _ in [[1],[3],[5],[7],[8],[-8..-1],[2]]]
cnm=[G2(a)*G2(b)/G2(a)/G2(b) for a in meridianos for b in meridianos]
w=G2(meridianos[-1])
print(meridianos)

[(1,), (3,), (4,), (5,), (6,), (-6, -5, 3, 1, -2, -1, -3, -4, 1, -2, -1, -3, -2, -1), (2,)]


We compute the derived subgroup (order $3$), we call $t$ a generator and we show its centrality (next three cells)

In [63]:
H=G2.subgroup(cnm)
H.order()

3

In [64]:
t=w*G2(meridianos[0])/w/G2(meridianos[0])
t.order()

3

In [67]:
control=True
for a in G2.generators():
    control=control and a*t/a/t==G2.one()
control

True

We check that the commutators of the meridians of the lines and the meridian of the cubic are of order $3$ and check that all are equal to $t$.

In [69]:
for a in meridianos[:-1]:
    a1=G2(a)
    c=w*a1/w/a1
    print(c.order(),c==t)

3 True
3 True
3 True
3 True
3 True
3 True


We check that the commutators of the meridians of the lines and the meridian of the cubic are of order $3$.

In [73]:
for a in meridianos[:-1]:
    a1=G1(a)
    c=w*a1/w/a1
    print(c.order(),c==t)

3 True
3 True
3 True
3 True
3 True
3 True


We need to check to which triangle are associated the meridians of the lines. 

In [72]:
for i,j in Combinations(6,2):
    a=G2(meridianos[i])
    b=G2(meridianos[j])
    u=a*b/a/b
    if u==G2.one():
        print(i,j)

0 1
0 3
0 5
1 2
1 4
2 3
2 5
3 4
4 5


Let us denote by $x_1,x_2,x_3$ the meridians corresponding to the places $0,2,4$ and by $z_1,z_2,z_3$ the meridians corresponding to the places $1,5,3$. We are going to check the presentation given in Proposition 4.3

In [79]:
x1,x2,x3=[G2(meridianos[j]) for j in (0,2,4)]
z1,z2,z3=[G2(meridianos[j]) for j in (1,5,3)]

In [75]:
control=True
for a,b in [(x1,x2),(x2,x3),(x3,x1)]:
    control=control and (a*b/a/b)==t
control

True

In [80]:
control=True
for a,b in [(z1,z2),(z2,z3),(z3,z1)]:
    control=control and (a*b/a/b)==t
control

True

In [81]:
control=True
for a in [x1,x2,x3]:
    for b in [z1,z2,z3]:
        control=control and (a*b/a/b).order()==1
control

True

In [82]:
control=True
for a in [x1,x2,x3,z1,z2,z3]:
    control=control and w*a/w/a==t
control

True

We have proved that $G_2$ has the following properties.
- It fits in a short exact sequence
$$
1\longrightarrow\mathbb{Z}/3\longrightarrow G_1\longrightarrow\left(\mathbb{Z}/3\right)^5\times\mathbb{Z}/9
\longrightarrow 1
$$
where the kernel is $G'_2$ and the extension is central and the cokernel is presented as the abelian group 
generated by $x_1,x_2,x_3,z_1,z_2,z_3,w$, where
$$
x_i^3=y_i^3=w^9=x_1\cdot x_2\cdot x_3\cdot z_1\cdot z_2\cdot z_3\cdot w=1.
$$
- It is generated by $x_1,x_2,x_3,z_1,z_2,z_3,w,t$ with the following relations:
    * $t^3=1$
    * $[t,x_i]=[t,z_i]=[t,w]=1$
    * $x_i^3=z_i^3=w^9=1$
    * $x_1\cdot x_2\cdot x_3\cdot z_1\cdot z_2\cdot z_3\cdot w=t$
    * $[x_1,x_2]=[x_2,x_3]=[x_3,x_1]=[z_1,z_2]=[z_2,z_3]=[z_3,z_1]=t$
    * $[x_i,z_j]=1$
    * $[w,x_i]=[w,z_j]=t$
    
The cells below end the proof.

In [83]:
control
for a in [x1,x2,x3,z1,z2,z3]:
    control=control and a^3==G2.one()
control

True

In [84]:
w^9==G2.one()

True

In [86]:
prod([x1,x2,x3,z1,z2,z3]+3*[w])==t

True